In [1]:
from parse_rules import *
import numpy as np
import pandas as pd
from py2neo import Node,Relationship,Graph,Path,Subgraph
from py2neo import NodeMatcher,RelationshipMatcher

import json

In [2]:
neo4j_url = 'http://localhost:7474/'
user = 'neo4j'
pwd = 'xmxujie13'
graph = Graph(neo4j_url,  auth=(user, pwd), name = "neo4j")
node_matcher = NodeMatcher(graph)

In [3]:
graph.run("MATCH (n) DETACH DELETE n")

### Read Data

In [ ]:
logFile = '../logs/auditbeat-20221130_filtered.json'

In [ ]:
# read
fr = open(logFile, "r")
logs = [json.loads(line) for line in fr.readlines()]
fr.close()


19025


In [ ]:
# 过滤只有program相关操作
filterLogs = []
filesLogs = []
netLogs = []

for log in logs:
    ruletype = RuleType().parse_jsons(log['rule'])
    
    if ruletype.tag == "sys_access":
        filesLogs.append(json.loads(log['log']))
        
    if ruletype.tag == "socket_as_client":
        netLogs.append(json.loads(log['log']))


### File Access Provenance graph
* Open File
* Read File
* Write File

In [ ]:
for log in filesLogs:
    
    pLocation = log['executable']
    
    if 'program' not in pLocation:
        continue
    
    fLocation = log['accessed_file']
    pName = pLocation.split('/')[-1]
    fName = fLocation.split('/')[-1]
    fType = 'File' if fName != 'secret.txt' else 'Secret'

    op = log['syscall']
    
    # additional information
    pid = log['pid'] 
    pUser = log['user']
    ts = log['timestamp']

    
    node1 = node_matcher.match("Process").where(name=pName).first()
    node2 = node_matcher.match("File").where(name=fName).first()
    
    if node1 is None:
        node1 = Node("Process", name=pName, location = pLocation, pid = pid, user = pUser['name'])
        graph.create(node1)
    
    if node2 is None:
        node2 = Node(fType, name=fName, location = fLocation)
        graph.create(node2)
        
    # build relationship
    relation1 = Relationship(node1, op, node2, timeStamp = ts)
    
    graph.create(relation1)
    

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'di

### NetWork Provenance Graph

In [ ]:
for log in netLogs:
    pLocation = log['executable']
    
    if 'program' not in pLocation:
        continue
    
    socket_addr = log['socket']
    pName = pLocation.split('/')[-1]
    ip = socket_addr['addr']
    port = socket_addr['port']
    protocol = socket_addr['family']
    
    op = log['syscall']
    
     # additional information
    pid = log['pid'] 
    pUser = log['user']
    ts = log['timestamp']

    node1 = node_matcher.match("Process").where(name = pName).first()
    node2 = node_matcher.match("NetWork").where(ip = ip).where(port = port).first()
    
    
    if node1 is None:
        node1 = Node("Process", name=pName, location = pLocation, pid = pid, user = pUser['name'])
        graph.create(node1)
    
    if node2 is None:
        node2 = Node("Network", name = "Host", ip = ip, port = port)
        graph.create(node2)
    
     # build relationship
    relation1 = Relationship(node1, op, node2, timeStamp = ts)
    
    graph.create(relation1)
    
    